In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel,LinearModel
from glob import glob

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
%matplotlib qt 

#Rutherfordstreuung

##Rutherfordsteuuformel
<br>
$\huge \huge \frac{d\sigma}{d\Omega} = 1.3 \cdot 10^{-3} \left( \frac{Z_1 Z_2}{E_0} \right) \frac{1}{sin^4\left(\frac{\theta}{2}\right)}$

<ul>
    <li>$\sigma$: Differentielle WQ
    <li>$\Theta$: Winkel
    <li>$Z_i$: Kernlagundszahl
    <li>E: Energie
</ul>   

In [3]:
def rutherfordsteuung(theta,Z_1,Z_2,E,b):
    e = 1.602 * 10**-19
    return (Z_1*Z_2*e**2)/(4*E*np.sin((3.14*theta-b)/360)**4)

#Energieverlust von $\alpha$-Strahlung in Luft

# Energie-Kanal kalibration

In [4]:
cd /home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/

/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib


In [5]:
data = np.array(pd.read_csv("energiekalpib_30Torr.csv",skiprows=15,delimiter="\t")).T

In [6]:
plt.errorbar(data[0],data[1],np.sqrt(data[1]))
plt.show()

## Händische schätzung für die Peaks

    - 1: x=250, y=180
    - 2: x=320, y=266
    - 3: x=373, y=158
    - 4: x=551, y=159
    
Mulitgauss mit den Werten konfigurieren und Fehler berechnen

In [7]:
err = np.sqrt(data[1][200:612])
err = np.where(err == 0, 1, err)

In [151]:
y = data[1][200:612]
x = data[0][200:612]


g_1 = GaussianModel(prefix="g_1")
pars = g_1.guess(y,x=x)

pars['g_1center'].set(260, min=240, max=275)
pars['g_1sigma'].set(15, min=3,max=22)
pars['g_1amplitude'].set(10000, min=90)

g_2 = GaussianModel(prefix="g_2")
pars.update( g_2.make_params())

pars['g_2center'].set(320, min=300, max=345)
pars['g_2sigma'].set(15, min=3)
pars['g_2amplitude'].set(20000, min=1)


g_3 = GaussianModel(prefix="g_3")
pars.update( g_3.make_params())

pars['g_3center'].set(373, min=350, max=400)
pars['g_3sigma'].set(15, min=3,max=20)
pars['g_3amplitude'].set(10000, min=1)

g_4 = GaussianModel(prefix="g_4")
pars.update( g_4.make_params())

pars['g_4center'].set(551, min=530, max=570)
pars['g_4sigma'].set(15, min=3)
pars['g_4amplitude'].set(20000, min=1)



mod = g_1 + g_2 + g_3 + g_4


out = mod.fit(y,pars,x=x,weights=1/err)

In [68]:
print(out.fit_report())

[[Model]]
    (((Model(gaussian, prefix='g_1') + Model(gaussian, prefix='g_2')) + Model(gaussian, prefix='g_3')) + Model(gaussian, prefix='g_4'))
[[Fit Statistics]]
    # function evals   = 166
    # data points      = 412
    # variables        = 12
    chi-square         = 686.653
    reduced chi-square = 1.717
[[Variables]]
    g_1center:      255.238186 +/- 0.978131 (0.38%) (init= 260)
    g_1sigma:       22         +/- 0.251185 (1.14%) (init= 15)
    g_1amplitude:   8700.66363 +/- 308.8438 (3.55%) (init= 10000)
    g_1fwhm:        51.8060400 +/- 0.591496 (1.14%)  == '2.3548200*g_1sigma'
    g_2center:      315.461920 +/- 0.556987 (0.18%) (init= 320)
    g_2sigma:       19.4956988 +/- 0.752410 (3.86%) (init= 15)
    g_2amplitude:   12483.3777 +/- 459.5527 (3.68%) (init= 20000)
    g_2fwhm:        45.9088615 +/- 1.771791 (3.86%)  == '2.3548200*g_2sigma'
    g_3sigma:       18.5261936 +/- 0.488221 (2.64%) (init= 15)
    g_3center:      372.541213 +/- 0.814979 (0.22%) (init= 373)
    

In [10]:
plt.errorbar(data[0][200:612],data[1][200:612],np.sqrt(data[1][200:612]),fmt='. b',label="Messdaten")
plt.plot(data[0][200:612],out.best_fit, '-r',label="Muli-Gauss-Fit")
plt.xlabel("Kanal",fontsize=22)
plt.ylabel('Counts',fontsize=22)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend()
plt.show()

## Mittelwerte den Energien zu ordnen
    - Peak  Energie  Kanal
    -   1    4,871   255+1
    -   2    5.49    315+0.6
    -   3    6.00    372+0.8
    -   4    7.69    549+0.3

In [11]:
eich_kanal = np.array([255,315,372,549])
eich_energie = np.array([4.871,5.49,6.00,7.69])
eich_errx = np.array([1,0.6,0.8,0.3])

In [12]:
mod_eich = LinearModel()
guess = mod_eich.guess(eich_energie,x=eich_kanal)
out_eich = mod_eich.fit(eich_energie,guess,x=eich_kanal)
print(out_eich.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 3
    # data points      = 4
    # variables        = 2
    chi-square         = 0.001
    reduced chi-square = 0.001
[[Variables]]
    intercept:   2.45818982 +/- 0.042991 (1.75%) (init= 2.45819)
    slope:       0.00953604 +/- 0.000111 (1.16%) (init= 0.009536043)
[[Correlations]] (unreported correlations are <  0.100)
    C(intercept, slope)          = -0.959 


In [13]:
plt.errorbar(eich_kanal,eich_energie,xerr=eich_errx,fmt='.r',label="Messdaten")
plt.plot(eich_kanal,out_eich.best_fit,label="linearer Fit")
plt.xlabel("Kanal",fontsize=22)
plt.ylabel('Energie [MeV]',fontsize=22)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlim([240,560])
plt.ylim([4.7,7.9])
plt.legend(loc="best")
plt.show()

## Konstanz der Zählrate

In [14]:
liste = []
for i in glob('/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/energieverlust/*.csv'):
    liste.append(np.array(pd.read_csv(i,skiprows=15,delimiter="\t")).T)

In [15]:
file_liste = []
liste = []
for i in glob('/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/energieverlust/*.csv'):
    file_liste.append(i)

file_liste.sort()

for file in file_liste:
    liste.append(np.array(pd.read_csv(file,skiprows=15,delimiter="\t")).T)

del(liste[20:25])

In [16]:
%matplotlib qt
for i in liste:
    plt.plot(i[0],i[1])
    plt.xlim([0,1000])
    plt.ylim([0,300])
    plt.show()

In [161]:
liste_fit_params_1 = []
liste_fit_params_2 = []
liste_fit_params_3 = []
liste_fit_params_4 = []

In [207]:

pars = g_4.guess(y,x=x)


g_4 = GaussianModel(prefix="g_4")
pars.update( g_4.make_params())


pars['g_4sigma'].set(15, min=3)
pars['g_4amplitude'].set(20000, min=1)



pars['g_4center'].set(124, min=0)

mod = g_4

In [225]:
i = liste[17]
out_2 = mod.fit(i[1][45:-5],pars,x=i[0][45:-5])
out_2.plot()
plt.show()
print(out_2.fit_report())

[[Model]]
    Model(gaussian, prefix='g_4')
[[Fit Statistics]]
    # function evals   = 38
    # data points      = 973
    # variables        = 3
    chi-square         = 8600.008
    reduced chi-square = 8.866
[[Variables]]
    g_4fwhm:        52.7057397 +/- 0.308114 (0.58%)  == '2.3548200*g_4sigma'
    g_4amplitude:   7077.06987 +/- 34.32181 (0.48%) (init= 20000)
    g_4center:      84.8697871 +/- 0.122480 (0.14%) (init= 124)
    g_4sigma:       22.3820673 +/- 0.130844 (0.58%) (init= 15)
[[Correlations]] (unreported correlations are <  0.100)
    C(g_4amplitude, g_4sigma)    =  0.631 
    C(g_4center, g_4sigma)       = -0.104 


In [223]:
liste_fit_params_4.append([])
liste_fit_params_4[-1].append(out_2.best_values["g_4center"])
liste_fit_params_4[-1].append(out_2.best_values["g_4amplitude"])
liste_fit_params_4[-1].append(out_2.best_values["g_4sigma"])

In [248]:
np_liste_fit_1


array([[  178.97013108,   151.30060604,   123.78831306,    96.57496267,
           67.75949842,    36.45335242],
       [ 9073.96441464,  8578.35503212,  8633.51729073,  8331.41029904,
         7843.46073328,  7858.70267292],
       [   22.        ,    21.99999434,    22.        ,    21.99999998,
           21.99999998,    21.99999999]])

In [259]:
t = 180
y_1 = np_liste_fit_1[1]/(np.sqrt(2*np.e)*np_liste_fit_1[2])
y_2 = np_liste_fit_2[1]/(np.sqrt(2*np.e)*np_liste_fit_2[2])
y_3 = np_liste_fit_3[1]/(np.sqrt(2*np.e)*np_liste_fit_3[2])
y_4 = np_liste_fit_4[1]/(np.sqrt(2*np.e)*np_liste_fit_4[2])

In [260]:
plt.plot(np.arange(100,len(y_1)*25+100,25),y_1/t,'r.')
plt.plot(np.arange(100,len(y_2)*25+100,25),y_2/t,'g.')
plt.plot(np.arange(100,len(y_3)*25+100,25),y_3/t,'b.')
plt.plot(np.arange(100,len(y_4)*25+100,25),y_4/t,'y.')
plt.show()

In [251]:
print(9073/(np.sqrt(2*np.e*22**2)))

176.874812006


In [256]:
np_liste_fit_1[1]/(np.sqrt(2*np.e)*np_liste_fit_1[2])

array([ 176.89361293,  167.23193258,  168.30725755,  162.41779255,
        152.90539444,  153.20253043])

# Energieverlust

In [265]:
x = 1.03 * np.arange(100,700,25)/760
x_delta = 1.03 * 4/760

In [323]:
print(np_liste_fit_1[0][0:-1]-np_liste_fit_1[0][1:])
print(x[1:len(np_liste_fit_1[1])]-x[0:len(np_liste_fit_1[1])-1])
print(np_liste_fit_2[0][0:-1]-np_liste_fit_2[0][1:])
print(np_liste_fit_3[0][0:-1]-np_liste_fit_3[0][1:])
print(np_liste_fit_4[0][0:-1]-np_liste_fit_4[0][1:])

[ 27.66952504  27.51229298  27.2133504   28.81546424  31.30614601]
[ 0.03388158  0.03388158  0.03388158  0.03388158  0.03388158]
[ 25.74613899  24.73453567  24.60520371  26.35492798  25.62359894
  28.95549868  27.63425321  17.65009846]
[ 24.58838439  22.89218354  22.74299087  23.38017192  22.56513977
  24.62030428  27.07742113  26.72880121  27.60936029  27.44575591
  12.97665456]
[ 23.21289228  22.02740894  19.99066487  21.32266751  20.49864123
  22.17525704  21.35591597  20.08700293  20.23255916  25.35053091
  19.8127719   21.6894296   21.8732437   21.14550144  22.32664177
  23.73968164  24.49593312  25.66296752  28.09907748]


In [339]:
e_1 = np_liste_fit_1[0][:-1]*0.01+2.46
e_2 = np_liste_fit_2[0][:-1]*0.01+2.46
e_3 = np_liste_fit_3[0][:-1]*0.01+2.46
e_4 = np_liste_fit_4[0][:-1]*0.01+2.46
e_x_1 = (np_liste_fit_1[0][0:-1]-np_liste_fit_1[0][1:])*0.01/(x[1:len(np_liste_fit_1[1])]-x[0:len(np_liste_fit_1[1])-1])
e_x_2 = (np_liste_fit_2[0][0:-1]-np_liste_fit_2[0][1:])*0.01/(x[1:len(np_liste_fit_2[1])]-x[0:len(np_liste_fit_2[1])-1])
e_x_3 = (np_liste_fit_3[0][0:-1]-np_liste_fit_3[0][1:])*0.01/(x[1:len(np_liste_fit_3[1])]-x[0:len(np_liste_fit_3[1])-1])
e_x_4 = (np_liste_fit_4[0][0:-1]-np_liste_fit_4[0][1:])*0.01/(x[1:len(np_liste_fit_4[1])]-x[0:len(np_liste_fit_4[1])-1])

In [341]:
plt.plot(e_1,e_x_1,'.r')
plt.plot(e_2,e_x_2,'.b')
plt.plot(e_3,e_x_3,'.g')
plt.plot(e_4,e_x_4,'.y')
plt.ylabel(r"$\frac{dE}{dx}$",fontsize=22)
plt.show()

In [299]:
(np_liste_fit_4[0][0:-1]-np_liste_fit_4[0][1:])*0.1

array([ 2.32128923,  2.20274089,  1.99906649,  2.13226675,  2.04986412,
        2.2175257 ,  2.1355916 ,  2.00870029,  2.02325592,  2.53505309,
        1.98127719,  2.16894296,  2.18732437,  2.11455014,  2.23266418,
        2.37396816,  2.44959331,  2.56629675,  2.80990775])

In [300]:
e = np.arange(0,1000)*0.1+2.46

In [305]:
e[:50]

array([ 2.46,  2.56,  2.66,  2.76,  2.86,  2.96,  3.06,  3.16,  3.26,
        3.36,  3.46,  3.56,  3.66,  3.76,  3.86,  3.96,  4.06,  4.16,
        4.26,  4.36,  4.46,  4.56,  4.66,  4.76,  4.86,  4.96,  5.06,
        5.16,  5.26,  5.36,  5.46,  5.56,  5.66,  5.76,  5.86,  5.96,
        6.06,  6.16,  6.26,  6.36,  6.46,  6.56,  6.66,  6.76,  6.86,
        6.96,  7.06,  7.16,  7.26,  7.36])